```python
# Bước 7: Tạo Trainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=1,  # Ghi log mỗi bước
)

trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate_fn,
)

trainer.train()



# Bước 8: Huấn luyện
trainer.train()
```

Lỗi 
```
Inputs being passed to tokenizer: [None, None, None, None, None, None, None, None]
---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-94-c1d51e013c93> in <cell line: 21>()
     19 )
     20 
---> 21 trainer.train()
     22 
     23 # Đánh giá trên tập kiểm tra

7 frames
<ipython-input-93-31b340d4e174> in collate_fn(features)
      7 
      8     # Đảm bảo tất cả các phần tử trong inputs là chuỗi
----> 9     assert all(isinstance(text, str) for text in inputs), "Some inputs are not strings!"
     10 
     11     labels = torch.tensor(labels, dtype=torch.long)

AssertionError: Some inputs are not strings!

```

- Check  mọi thứ bình thường ko biết lỗi ở đâu: 

```python 
from torch.utils.data import DataLoader

# Kiểm tra DataLoader để đảm bảo không có None trong các batch
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    collate_fn=collate_fn
)

for batch in train_loader:
    print(batch)
    break  # In thử một batch đầu tiên

```

```
Inputs being passed to tokenizer: ['Tôi không chắc, hãy để tôi kiểm tra lại.', 'Bạn có thể nhắc lại không?']
{'input_ids': tensor([[  101,  2000,  2072,  1047, 19991, 15775,  2278,  1010, 10974,  1102,
          2063,  2000,  2072, 11382,  6633, 19817,  2050, 21110,  1012,   102],
        [  101,  7221,  2522,  1996, 18699,  6305, 21110,  1047, 19991,  1029,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([2, 0])}
```

```python 
# Lấy batch từ DataLoader
for batch in train_loader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    break

# Chuyển batch sang thiết bị (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

# Chạy qua model
model = model.to(device)
outputs = model(input_ids=input_ids, attention_mask=attention_mask)
print("Model outputs:", outputs)

```

```
Inputs being passed to tokenizer: ['Tôi không chắc, hãy để tôi kiểm tra lại.', 'Bạn có thể nhắc lại không?']
Model outputs: tensor([[ 0.1334, -0.1960, -0.1488, -0.3732, -0.3232],
        [ 0.3185,  0.1669, -0.2354, -0.4047, -0.3181]],
       grad_fn=<AddmmBackward0>)
```

### ? 

```
===== Một số mẫu trong DataFrame =====
                                       text           intent
0     Vâng, tôi muốn cho bạn xem bức tranh.           Đồng ý
1                    Không, tôi không muốn.          Từ chối
2                           Tôi không biết.  Không chắc chắn
3  Xin lỗi, tôi không hiểu bạn đang nói gì.         Fallback
4             Được rồi, để tôi cho bạn xem.           Đồng ý

===== Kiểm Tra Giá Trị Không Hợp Lệ Trong Cột 'text' =====
Số giá trị None: 0
Số giá trị rỗng: 0
===== Features của Dataset =====
{'text': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None)}
===== Một số mẫu trong Dataset =====
{'text': 'Vâng, tôi muốn cho bạn xem bức tranh.', 'label': 'Đồng ý'}
{'text': 'Không, tôi không muốn.', 'label': 'Từ chối'}
{'text': 'Tôi không biết.', 'label': 'Không chắc chắn'}
{'text': 'Xin lỗi, tôi không hiểu bạn đang nói gì.', 'label': 'Fallback'}
{'text': 'Được rồi, để tôi cho bạn xem.', 'label': 'Đồng ý'}

===== Invalid Samples After Preprocessing =====
[]
Label mapping: {'Fallback': 0, 'Im lặng': 1, 'Không chắc chắn': 2, 'Từ chối': 3, 'Đồng ý': 4}
Số lượng lớp (num_classes): 5
Map: 100%
 21/21 [00:00<00:00, 858.58 examples/s]
Map: 100%
 6/6 [00:00<00:00, 267.12 examples/s]
===== Một số mẫu trong Train Dataset sau khi chuyển đổi nhãn =====
{'text': 'Tôi không chắc, hãy để tôi kiểm tra lại.', 'label': 2}
{'text': 'Bạn có thể nhắc lại không?', 'label': 0}
{'text': 'Hoàn toàn chính xác, để tôi làm.', 'label': 4}
{'text': 'Không, điều đó không đúng.', 'label': 3}
{'text': 'Chắc chắn rồi, để tôi làm.', 'label': 4}
===== Một số mẫu trong Test Dataset sau khi chuyển đổi nhãn =====
{'text': 'Tôi chưa hiểu rõ câu hỏi của bạn.', 'label': 0}
{'text': 'Không có gì để nói cả.', 'label': 1}
{'text': 'Để tôi xem xét thêm.', 'label': 2}
{'text': 'Tôi chưa quyết định.', 'label': 2}
{'text': 'Không, tôi không đồng ý.', 'label': 3}
tokenizer_config.json: 100%
 48.0/48.0 [00:00<00:00, 3.77kB/s]
config.json: 100%
 570/570 [00:00<00:00, 46.5kB/s]
vocab.txt: 100%
 232k/232k [00:00<00:00, 6.26MB/s]
tokenizer.json: 100%
 466k/466k [00:00<00:00, 7.35MB/s]

===== Model Info =====
BERTIntentClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermediate_act_fn): GELUActivation()
          )
          (output): BertOutput(
            (dense): Linear(in_features=3072, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (pooler): BertPooler(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (activation): Tanh()
    )
  )
  (ffnn): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=768, out_features=5, bias=True)
  )
)
Số lượng lớp (num_classes): 5

===== Tokenizer Output =====
Input IDs: tensor([[  101,  8418,  2078, 22455,  1010,  7221,  1047, 14490,  1047, 19991,
          1029,   102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

===== Forward Pass Output =====
Logits shape: torch.Size([1, 5])
Logits: tensor([[ 0.3990,  0.3353, -0.5161, -0.2652, -0.4595]])
Probabilities: tensor([[0.3051, 0.2863, 0.1222, 0.1570, 0.1293]])
Predicted class: 0
Loss: 2.0455498695373535
<ipython-input-98-10abfcead99b>:254: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerCustom.__init__`. Use `processing_class` instead.
  trainer = TrainerCustom(
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 1}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 3}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
 [9/9 00:17, Epoch 3/3]
Epoch	Training Loss	Validation Loss
1	No log	No log
2	No log	No log
3	No log	No log
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 4}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 3}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 2}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 4}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 2}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 3}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 3}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 4}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 4}
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 0}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 3}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
Sample with invalid text encountered: {'label': 0}
Sample with invalid text encountered: {'label': 1}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 2}
Sample with invalid text encountered: {'label': 3}
Sample with invalid text encountered: {'label': 3}
Inputs being passed to tokenizer: ['EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT', 'EMPTY_TEXT']
 [1/1 : < :]
{'eval_runtime': 0.1329,
 'eval_samples_per_second': 45.15,
 'eval_steps_per_second': 7.525,
 'epoch': 3.0}
1
Beta
0 / 0
used queries
```